In [5]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

import base64

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUDModule import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

waterBreeds = ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']
mountainBreeds = ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']
disasterBreeds = ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']

def create_filtered_df(breeds, sex, age_min, age_max):
    return df[
        (df['breed'].isin(breeds)) &
        (df['sex_upon_outcome'] == sex) &
        (df['age_upon_outcome_in_weeks'] >= age_min) &
        (df['age_upon_outcome_in_weeks'] <= age_max)
    ]

# Initial dataframes
water_df = create_filtered_df(waterBreeds, 'Intact Female', 26, 156)
mountain_df = create_filtered_df(mountainBreeds, 'Intact Male', 26, 156)
disaster_df = create_filtered_df(disasterBreeds, 'Intact Male', 20, 300)

all_counts = df['breed'].value_counts().reset_index()
all_counts.columns = ['breed', 'count']

# Pie chart creation function takes filtered dataframes and name of pie chart as input
def create_pie_chart(df, title, is_pre_aggregated=False):
    if not is_pre_aggregated: # had to include this due to the initial write of the code leading to the def. chart not working as intended
        counts = df['breed'].value_counts().reset_index()
        counts.columns = ['breed', 'count']
        
    else:
            counts = df
        
    return px.pie(counts, names='breed', values='count', title=title)

initial_pie_chart = create_pie_chart(all_counts, 'Austin Animal Rescue')



## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard: Weaver'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Hr(),
    dcc.RadioItems(
    id='Quick Search Rescue Breeds:',
    options =[
        {'label': 'Water Rescue', 'value': '1'},
        {'label': 'Mountain Rescue', 'value': '2'},
        {'label': 'Disaster Rescue', 'value': '3'},
        {'label': 'Reset', 'value': '4'}
    ],
        value='4' #Default Data Table setting
    ),
    
    html.Hr(style={'marginTop': '20px', 'marginBottom': '20px'}),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        cell_selectable = True,
        row_selectable='single',
        selected_rows=[],
        sort_action="native",
        filter_action="native",
        page_size=10,
        style_table={'overflow': 'auto'},

    ),
    html.Br(),
     html.Hr(),
    
    #pie chart
    dcc.Graph(id='pie-chart', figure=initial_pie_chart),
    
    html.Br(),
    html.Hr(),
    
    html.Div(id='output-container'),
    
    #map
     html.Div(
            id='map-id',
            className='col s12 m6',
            )
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    [Output('pie-chart', 'figure'),
    Output('datatable-id', 'data')],
    [Input('Quick Search Rescue Breeds:', 'value')]
)
def update_content(selected_value):
    if selected_value == '1': #water rescue
        df = water_df
        title = 'Water Rescue'
        is_pre_aggregated = False
    elif selected_value == '2': #mountain rescue
        df = mountain_df
        title = 'Mountain Rescue'
        is_pre_aggregated = False
    elif selected_value == '3': #disaster rescue
        df = disaster_df
        title = 'Disaster Rescue'
        is_pre_aggregated = False
    else: #default/reset
        df = all_counts # Everything!? it works as intended just looks... well bad
        title = 'Animals'
        is_pre_aggregated = True
        
    pie_chart = create_pie_chart(df, title, is_pre_aggregated)
    return pie_chart, df.to_dict('records')


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
 # Because we only allow single row selection, the list can 
 # be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]

# Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
              children=[
              dl.Tooltip(dff.iloc[row,4]),
              dl.Popup([
                 html.H1("Animal Name"),
                html.P(dff.iloc[row,9])
             ])
          ])
       ])
    ]
    
if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:13698/
